In [1]:
import pandas as pd
import glob
import sys
import glob
sys.path.append('/home/pierre/riken/io')

from riken.protein_io import reader

pd.options.display.max_colwidth = 50

## Allergens

In [2]:
allergens_df = reader.read_fasta('/home/pierre/riken/data/riken_data/COMPARE_2018.txt').assign(is_allergenic=lambda x: True)

genre_species = (allergens_df.description.str.extract(r'\[(.*?)\]').iloc[:, 0]
                         .str.lower()
                         .str.split(expand=True).iloc[:, :2])

prot_index = (allergens_df.description
                         .str.split(pat='|', expand=True).iloc[:, 1]
                         .str.lower())

allergens_df.loc[:, 'genre'], allergens_df.loc[:, 'species'] = genre_species.iloc[:, 0], genre_species.iloc[:, 1]
allergens_df.loc[:, 'prot_index'] = prot_index

In [3]:
allergens_df.head(2)

,description,idx,name,sequences,is_allergenic,genre,species,prot_index
0,accession|2|Putative Blo t 1.02 Manual Entry [...,accession|2|Putative,accession|2|Putative,"(M, K, F, L, L, V, A, A, L, C, A, L, V, A, I, ...",True,blomia,tropicalis,2
1,accession|3|Putative vitellogenin I Manual Ent...,accession|3|Putative,accession|3|Putative,"(E, I, A, S, Q, I, A, Q, E, D, Q, S, T, C, E, ...",True,gallus,gallus,3


## Non-allergens

In [4]:
no_allergens_path = glob.glob('/home/pierre/riken/data/riken_data/xlsx/*')
excel_df = pd.concat([pd.read_excel(file, sheet_name='non allergen') for file in no_allergens_path], 
                     ignore_index=True)
aa_sequences_path = glob.glob('/home/pierre/riken/data/riken_data/fasta/*')
seq_df = (pd.concat([reader.read_fasta(file) for file in aa_sequences_path], ignore_index=True)
          .assign(Entry=lambda x: x.name.str.split(pat='|', expand=True)[1]))

In [5]:
seq_df.head(2)

,description,idx,name,sequences,Entry
0,sp|I1JLC8|SLE2_SOYBN Protein SLE2 OS=Glycine m...,sp|I1JLC8|SLE2_SOYBN,sp|I1JLC8|SLE2_SOYBN,"(M, A, S, R, Q, N, N, K, Q, E, L, D, E, R, A, ...",I1JLC8
1,sp|Q39821|SDLCA_SOYBN Dynamin-related protein ...,sp|Q39821|SDLCA_SOYBN,sp|Q39821|SDLCA_SOYBN,"(M, E, N, L, I, S, L, V, N, K, I, Q, R, A, C, ...",Q39821


In [6]:
excel_df.head(2)

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length
0,A2VDL6,AT1A2_BOVIN,reviewed,Sodium/potassium-transporting ATPase subunit a...,ATP1A2,Bos taurus (Bovine),1020
1,Q3T0C6,AT1B3_BOVIN,reviewed,Sodium/potassium-transporting ATPase subunit b...,ATP1B3,Bos taurus (Bovine),279


In [7]:
non_allergens_df = (pd.merge(seq_df, excel_df, on='Entry')
                        .rename({"Entry": 'prot_index'}, axis=1)
                        .assign(is_allergenic=lambda x: False, 
                                prot_index=lambda x: x.prot_index.str.lower()))

In [8]:
genre_species = non_allergens_df.Organism.str.lower().str.split(expand=True).iloc[:, :2]
non_allergens_df.loc[:, 'genre'], non_allergens_df.loc[:, 'species'] = genre_species.iloc[:, 0], genre_species.iloc[:, 1]

In [9]:
non_allergens_df.sample()

,description,idx,name,sequences,prot_index,Entry name,Status,Protein names,Gene names,Organism,Length,is_allergenic,genre,species
4132,sp|Q0P5B1|PEX13_BOVIN Peroxisomal membrane pro...,sp|Q0P5B1|PEX13_BOVIN,sp|Q0P5B1|PEX13_BOVIN,"(M, A, S, Q, P, P, P, P, P, K, P, W, E, T, R, ...",q0p5b1,PEX13_BOVIN,reviewed,Peroxisomal membrane protein PEX13 (Peroxin-13),PEX13,Bos taurus (Bovine),403,False,bos,taurus


## Merging Data

In [10]:
COLS_TO_KEEP = ['is_allergenic', 'sequences', 'genre', 'species', 'prot_index']

In [11]:
non_allergens_df = non_allergens_df.loc[:, COLS_TO_KEEP]
allergens_df = allergens_df.loc[:, COLS_TO_KEEP]

df = pd.concat((allergens_df, non_allergens_df), ignore_index=True) \
                .assign(seq_len=lambda x: x.sequences.apply(len))

In [15]:
import numpy as np

a = pd.DataFrame({"name": [1, 2, 3], 'val': ["aaada", "adlams", 'dlad']})
a.val + np.array(['1', '2', '3'])

0     aaada1
1    adlams2
2      dlad3
Name: val, dtype: object

In [18]:
def rename_prot_index(dataf):
    if len(dataf) <= 1:
        return dataf
    else:
        renamer = np.array(['_'+str(inte) for inte in range(len(dataf))])
        dataf.loc[:, 'prot_index'] = dataf.prot_index + renamer
        return dataf

df = (df.groupby('prot_index').apply(rename_prot_index).reset_index(drop=True))

In [22]:
df.to_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx_v2.tsv', sep='\t', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12649 entries, 0 to 12648
Data columns (total 6 columns):
is_allergenic    12649 non-null bool
sequences        12649 non-null object
genre            12647 non-null object
species          12644 non-null object
prot_index       12649 non-null object
seq_len          12649 non-null int64
dtypes: bool(1), int64(1), object(4)
memory usage: 506.5+ KB


In [12]:
nb_duplicates = df.groupby('sequences').size().to_frame('nb_identical')
df = pd.merge(df, nb_duplicates, left_on='sequences', right_index=True)
problems = df[df.nb_identical >= 2]

/home/pierre/.conda/envs/python3/lib/python3.5/site-packages/Bio/Seq.py:163: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [13]:
problems[problems.is_allergenic].shape

(12, 7)

In [55]:
problems[~problems.is_allergenic].shape

(212, 7)

# OLD - Are Both files the same

In [46]:
v1 = pd.read_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx.tsv', sep='\t')
v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12851 entries, 0 to 12850
Data columns (total 5 columns):
is_allergenic    12851 non-null bool
sequences        12851 non-null object
genre            12849 non-null object
species          12846 non-null object
seq_len          12851 non-null int64
dtypes: bool(1), int64(1), object(3)
memory usage: 414.2+ KB


In [59]:
# v1 = pd.read_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx.tsv', sep='\t')
# v2 = pd.read_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx_with_index.tsv', sep='\t')

# should_be_eq = ['sequences', 'is_allergenic', 'genre', 'species']

# v1_vals = v1[should_be_eq].values
# v2_vals = v2[should_be_eq].values

# v1_vals = set([str(row) for row in v1_vals])
# v2_vals = set([str(row) for row in v2_vals])

# assert len(v1_vals.difference(v2_vals)) == 0
# assert len(v2_vals.difference(v1_vals)) == 0

In [75]:
nb_duplicates = v2.groupby('sequences').size().to_frame('nb_identical')
df = pd.merge(v2, nb_duplicates, left_on='sequences', right_index=True)
problems = df[df.nb_identical >= 2]

In [64]:
v1[v1.is_allergenic].shape

(2038, 5)